In this approach we will use document metadata: named entities and keywords<br>
    - We will read corpus_with_parties_NE_24.10.2022.csv<br>
    - We will read keywords.csv

In [ ]:
import pandas as pd
df = pd.read_csv('../dataset/corpus_with_parties_NE_24.10.2022.csv')

In [ ]:
kwd_df = pd.read_csv('../dataset/keywords.csv')
kwd_df=kwd_df.iloc[: , 1:]
kwd_df.head(3)

In [ ]:
df = df.merge(kwd_df, on='id')
df.head(1)

In [ ]:
# Convert df['NE'] and df['keywords'] to python List
from ast import literal_eval

features=['NE', 'keywords']

for feature in features:
    df[feature] = df[feature].astype(str)
    df[feature]=df[feature].apply(literal_eval)
df['keywords'].head(3)

In [ ]:
# Function to sanitize data
def sanitize(ls):
    if isinstance(ls, list):
        x = [i for i in ls if len(i) > 1]
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "").replace(".","")) for i in x]
    return ''

df['NE'] = df['NE'].apply(sanitize)
print(df['NE'].iloc[0])

In [ ]:
#merge NE with keywords in one column and remove duplicates
df=df.reset_index(drop=True)

SOUP=[]
for index, row in df.iterrows():
    soup_list=[]
    soup_list=soup_list+row['NE']+row['keywords']
    SOUP.append(soup_list)

df['KWD_NE']=SOUP

#remove duplicates
def remove_duplicates(mlist):
    return list(dict.fromkeys(mlist))

df['KWD_NE'] = df['KWD_NE'].apply(remove_duplicates)
print(df['KWD_NE'].iloc[0])

In [ ]:
#Create soup of NEs
# df['soup'] = df['NE'].apply(lambda x: (' '.join(x)))
# df['soup']

#Create soup of df[KWD_NE]
df['soup'] = df['KWD_NE'].apply(lambda x: (' '.join(x)))
df['soup'].head(3)


WE will compute cosine similarity among df['soup'] items

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df = df.reset_index(drop=True)
indices2 = pd.Series(df.index, index=df['title'])

In [ ]:
def content_recommender(title, cosine_sim, df, indices):
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]

    return df.iloc[article_indices]

In [26]:
#Republiacn
df_repub=content_recommender('Republicans aim to turn congressional wins into seats after redistricting', cosine_sim2, df, indices2)
df_democ=content_recommender('Democrats renew push for immigration bill', cosine_sim2, df, indices2)

df_all=pd.concat([df_repub,df_democ])
df_all[['title','parties']]
#The first 10 rows is for Republican, and the second 10 rows is for Democrats

,title,parties
388,Republicans who decried Obamacare secrecy now writing legislation in secret,['Republicans']
906,Democrats have a depth problem,['Democrats']
878,Republicans and Democrats agree: Their own member of Congress isn’t part of the problem,"['Democrats', 'Republicans']"
43,"Shutdown means damage to the system, but greater risk for Republicans",['Republicans']
787,House Democrats need 27 signatures to force vote on comprehensive immigration reform bill,['Democrats']
1767,Republicans believe these 10 House incumbents will face the toughest campaigns of 2018,['Republicans']
1903,House GOP leaders warn Republicans about 2018: Watch out,['Republicans']
1798,House Republicans set fundraising record in President Trump’s first full month,['Republicans']
561,Why Republicans shouldn’t filibuster the gun bill,['Republicans']
1056,Early Census data shows electoral college advantage for Republicans,['Republicans']


In [ ]:
df_all = df_all.iloc[: , 2:]

In [ ]:
df_all

In [ ]:
df_all.to_csv('output/personalized_all_columns.csv')

#remove last 4 columns to fit the databas
output_df = df_all.iloc[: , :-6]
output_df.to_csv('output/personalized_db.csv')